### 0 - Caucasian
### 1 - Indian
### 2 - Asian
### 3 - Asian

In [1]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models
from cosface import CosFaceLoss

In [20]:
# Load data from directory structure using ImageFolder
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = ImageFolder(root='/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/resources/data/train/', transform=transform)
train_loader = DataLoader(train_data, batch_size=512, shuffle=True)

# Initialize ResNet34 model
model = models.resnet34(pretrained=False)
num_ftrs = model.fc.in_features
num_classes = len(train_data.classes)
model.fc = nn.Linear(num_ftrs, num_classes)
model = model.cuda() if torch.cuda.is_available() else model

print(100 * "*")
print("Number of Classes: ", num_classes)
print(100 * "*")
print("Features dimension: ", num_ftrs)
print(100 * "*")
print(model.fc)
print(100 * "*")
print(model)
print(100 * "*")

/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/.venv/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


****************************************************************************************************
Number of Classes:  4
****************************************************************************************************
Features dimension:  512
****************************************************************************************************
Linear(in_features=512, out_features=4, bias=True)
****************************************************************************************************
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affi

In [21]:
# Initialize CosFace loss function
criterion = CosFaceLoss(in_features=num_ftrs, out_features=num_classes, s=64.0, m=0.35)

# Initialize SGD optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Learning rate scheduler with step decay
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[8, 18, 30, 34], gamma=0.1)

In [19]:
import os

data_path = "/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/resources/newdata"
# race, identity, name, visual_features

items = []

races = os.listdir(data_path)

for race in races:
    if race != '.DS_Store':
        path_with_race = os.path.join(data_path, race)

        identity_list = os.listdir(path_with_race)

        for identity in identity_list:

            if identity != '.DS_Store':
                path_with_identity = os.path.join(path_with_race, identity)

                image_list = os.listdir(path_with_identity)

                for image in image_list:

                    image_with_path = os.path.join(path_with_identity, image)

                    visual_features = extract_features(image_with_path)

                    mydict = {
                        'race': race,
                        'identity': identity,
                        'name': image.split('.')[0],
                        'visual_features': visual_features
                    }

                    items.append(mydict)

In [ ]:
import pandas as pd

df = pd.DataFrame(items)
df.to_excel('/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/resources/smallbalancefacedata/ipr200_df.xlsx', index=False)

In [27]:
%pip install pandas
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd

ipr200_df = pd.read_excel('/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/resources/smallbalancefacedata/200i_per_race.xlsx')

ipr200_df.head()

,race,identity,name,visual_features
0,Indian,m.0h1gqql,000077_00@en,[ 7.22792372e-02 -3.69315267e-01 1.88261241e-...
1,Indian,m.0h1gqql,000125_00@en,[-0.00465955 -0.39388263 0.36671907 -0.040767...
2,Indian,m.0h1gqql,000000_00@en,[ 1.50705129e-02 -4.23007339e-01 1.46263987e-...
3,Indian,m.0h1gqql,000086_00@en,[ 6.24053925e-02 -4.20565814e-01 1.52196497e-...
4,Indian,m.0h1gqql,000035_00@en,[ 1.50490552e-01 -4.06864762e-01 3.23215187e-...


In [6]:
ipr200_df.shape

(14854, 4)

In [7]:
ides_df = ipr200_df.groupby('identity')['name'].count().reset_index()
ides_df.head()

,identity,name
0,m.01dhg,20
1,m.01f6nq,88
2,m.01ggxh,23
3,m.01h7zl,45
4,m.01j0183,61


In [132]:
# def getIdentityPairs(grouped_by_identity):

#     lis = []
#     final_list = []

#     for identity, identity_group in grouped_by_identity:
#         new_list = []
#         for other_identity, other_identity_group in grouped_by_identity:

#             if identity != other_identity:

#                 name1 = list(identity_group['name'])
#                 name2 = list(other_identity_group['name'])
                
#                 # Forming pairs and adding them to the list
#                 for n1 in name1:
#                     for n2 in name2:
#                         if n1 != n2:
#                             pair = sorted([n1, n2])
#                             if pair not in lis:
#                                 lis.append(pair)
#                                 d = {
#                                     'name1': pair[0],
#                                     'name2': pair[1]
#                                 }

#                                 new_list.append(d)
#         for d in new_list:
#             d['identity'] = identity
        
#         final_list.append(new_list)
    
#     return final_list

In [134]:
# import pandas as pd

# # Group the DataFrame by race
# grouped_by_race = ipr200_df.groupby('race')

# # Create an empty list to store dictionaries representing pairs
# pairs_list = []

# # Iterate over each race group
# for race, race_group in grouped_by_race:
#     # Group by identity within the race
#     grouped_by_identity = race_group.groupby('identity')

#     finalpairlist = getIdentityPairs(grouped_by_identity)
#     print(len(finalpairlist))
#     break

# # Create a DataFrame from the list of dictionaries
# pairs_df = pd.DataFrame(pairs_list)
# pairs_df.head()

# # ('11-FaceId-0_align', '62-FaceId-0_align')

KeyboardInterrupt: 

In [8]:
import itertools

def get_identity_pairs(grouped_by_identity):
    pairs_list = []
    for identity, identity_group in grouped_by_identity:
        for other_identity, other_identity_group in grouped_by_identity:
            if identity != other_identity:
                name1 = set(identity_group['name'])
                name2 = set(other_identity_group['name'])
                # Forming pairs and adding them to the list
                for n1, n2 in itertools.product(name1, name2):
                    if n1 != n2:
                        pair = tuple(sorted([n1, n2]))
                        pairs_list.append({'name1': pair[0], 'name2': pair[1], 'identity': identity})
    return pairs_list

import pandas as pd

# Group the DataFrame by race
grouped_by_race = ipr200_df.groupby('race')

# Create an empty list to store dictionaries representing pairs
pairs_list = []

# Iterate over each race group
for race, race_group in grouped_by_race:
    # Group by identity within the race
    grouped_by_identity = race_group.groupby('identity')
    final_pair_list = get_identity_pairs(grouped_by_identity)
    pairs_list.extend(final_pair_list)

# Create a DataFrame from the list of dictionaries
pairs_df = pd.DataFrame(pairs_list)
pairs_df.head()




,name1,name2,identity
0,11-FaceId-0_align,99-FaceId-0_align,m.01dhg
1,11-FaceId-0_align,16-FaceId-0_align,m.01dhg
2,11-FaceId-0_align,79-FaceId-0_align,m.01dhg
3,11-FaceId-0_align,94-FaceId-0_align,m.01dhg
4,11-FaceId-0_align,95-FaceId-0_align,m.01dhg


In [1]:
pairs_df.shape

NameError: name 'pairs_df' is not defined

In [ ]:
print(ipr200_df.columns)
print(pairs_df.columns)

In [ ]:
# The End

In [19]:
image_vector1 = extract_features('C:\\Users\\student\\Desktop\\D K\\Fairness-Image-Recognization\\000005_00@.jpg')
image_vector2 = extract_features('C:\\Users\\student\\Desktop\\D K\\Fairness-Image-Recognization\\000006_00@.jpg')


In [21]:
image_vector1

(array([ 6.46108866e-01,  9.07606930e-02, -4.40394461e-01,  1.52964365e+00,
        -1.92985922e-01, -2.87636183e-02,  1.04131043e+00,  2.76630819e-01,
         9.07576531e-02, -2.50827491e-01,  2.67229021e-01,  4.03658263e-02,
        -5.12113512e-01, -7.23843992e-01,  2.42572412e-01,  3.42598945e-01,
         4.12478238e-01,  5.19691169e-01, -1.49934128e-01,  9.07510519e-01,
         3.65649648e-02, -7.28801370e-01, -1.60490766e-01,  5.06774664e-01,
         2.15909526e-01, -4.96059120e-01,  1.09126732e-01, -3.56301785e-01,
        -1.62118658e-01, -1.03738494e-01,  4.21938777e-01, -8.42407942e-01,
         2.36395478e-01, -1.97045475e-01, -9.20296609e-01, -5.16177773e-01,
         5.42731024e-02,  4.97454494e-01,  8.00171047e-02,  2.29755893e-01,
        -1.24321568e+00, -3.09921831e-01,  3.31916869e-01,  4.35812950e-01,
        -8.01793277e-01, -5.99602342e-01,  8.06619704e-01,  2.31111050e-01,
         6.11216605e-01, -1.00014091e-01, -8.59280303e-02,  2.54619867e-01,
        -5.2

In [22]:
image_vector2

array([ 6.56450629e-01,  3.95283848e-02, -2.50434220e-01,  1.21283817e+00,
       -1.08747810e-01, -7.49328583e-02,  9.45998907e-01,  2.68246830e-01,
       -4.25798893e-02, -5.62191606e-02,  2.72962153e-01,  6.81514740e-02,
       -3.22477758e-01, -5.42581558e-01,  2.67338306e-01,  2.58857280e-01,
        3.08022887e-01,  4.78530616e-01, -9.85497385e-02,  7.76668310e-01,
        4.89973053e-02, -4.64418620e-01, -1.75289750e-01,  4.42366719e-01,
        1.91827163e-01, -4.42653298e-01,  1.63919002e-01, -3.73374820e-01,
       -2.15909541e-01, -1.24035791e-01,  3.57463092e-01, -6.50598764e-01,
        1.09132789e-01, -1.92116648e-01, -6.75803006e-01, -3.60896468e-01,
        6.73876107e-02,  3.60271245e-01,  1.38571173e-01,  1.82644218e-01,
       -9.06132579e-01, -2.24162325e-01,  2.35309526e-01,  4.03030366e-01,
       -6.32650197e-01, -4.26654160e-01,  7.72287428e-01,  6.26419783e-02,
        4.94167238e-01, -2.06612833e-02, -9.25800875e-02,  2.15091050e-01,
       -3.36828411e-01, -

In [24]:
# import numpy as np
# from __future__ import print_function
# from __future__ import division
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.nn import Parameter
# import math

# def cosine_sim(x1, x2, dim=1, eps=1e-8):
#     ip = torch.mm(x1, x2.t())
#     w1 = torch.norm(x1, 2, dim)
#     w2 = torch.norm(x2, 2, dim)
#     return ip / torch.ger(w1,w2).clamp(min=eps)

In [27]:
import numpy as np
import torch

# Define cosine similarity function
def cosine_sim(x1, x2, dim=1, eps=1e-8):
    x1_tensor = torch.tensor(x1).unsqueeze(0)  # Add a batch dimension
    x2_tensor = torch.tensor(x2).unsqueeze(0)  # Add a batch dimension
    ip = torch.mm(x1_tensor, x2_tensor.t())
    w1 = torch.norm(x1_tensor, 2, dim)
    w2 = torch.norm(x2_tensor, 2, dim)
    return ip / torch.ger(w1, w2).clamp(min=eps)

# Assuming image_vector1 and image_vector2 are NumPy arrays
image_vector1 = np.array(image_vector1)
image_vector2 = np.array(image_vector2)

# Calculate cosine similarity
similarity = cosine_sim(image_vector1, image_vector2, dim=1, eps=1e-8)
print(similarity)


tensor([[0.9746]])


In [28]:
image_vector3 = extract_features("C:\\Users\\student\\Desktop\\D K\\Fairness-Image-Recognization\\000002_00@ja.jpg")

In [29]:
image_vector3

array([ 3.36448014e-01,  2.44794920e-01, -2.04259485e-01,  8.09584796e-01,
       -8.39712322e-02, -8.60218704e-02,  8.68313670e-01,  5.70964888e-02,
       -2.20754147e-01, -1.63755357e-01,  3.69698703e-01,  4.68951277e-02,
       -3.71781707e-01, -5.19027412e-01,  3.46580535e-01,  1.34132400e-01,
        4.70498316e-02,  3.35919976e-01,  1.68948472e-02,  7.79271901e-01,
        6.14125803e-02, -2.36297458e-01, -1.37879446e-01,  3.97197902e-01,
        2.13120267e-01, -3.62892836e-01,  3.40588093e-01, -3.52750301e-01,
       -2.58135349e-01, -1.62466586e-01,  3.31187099e-01, -4.45414245e-01,
       -1.52858198e-02, -2.90845007e-01, -3.57043356e-01, -2.85990000e-01,
        2.36561462e-01,  2.24748135e-01,  8.81955475e-02,  8.85408968e-02,
       -4.19519454e-01, -2.79714972e-01,  1.64138913e-01,  8.85270610e-02,
       -4.11653727e-01, -3.93951833e-01,  4.51094955e-01, -2.67285794e-01,
        5.61222434e-01,  5.40339574e-03, -5.00315800e-02,  3.27177912e-01,
       -6.48868829e-02, -

In [30]:
similarity1 = cosine_sim(image_vector1, image_vector3, dim=1, eps=1e-8)
print(similarity1)

tensor([[0.8588]])


In [31]:
image_vector4 = extract_features("C:\\Users\\student\\Desktop\\D K\\Fairness-Image-Recognization\\8-FaceId-1_align.jpg")

In [38]:
similarity2 = cosine_sim(image_vector2, image_vector4, dim=1, eps=1e-8)
print(similarity2)

tensor([[0.9502]])


In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Define your MarginCosineProduct class
class MarginCosineProduct(nn.Module):
    r"""Implement of large margin cosine distance: :
    Args:
        in_features: size of each input sample
        out_features: size of each output sample
        s: norm of input feature
        m: margin
    """

    def __init__(self, in_features, out_features, s=64.0, m=0.35):
        super(MarginCosineProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, input, label):
        cosine = F.normalize(input) @ F.normalize(self.weight).T
        one_hot = F.one_hot(label, self.out_features)
        output = self.s * (cosine - one_hot * self.m)
        return output

    def __repr__(self):
        return self.__class__.__name__ + '(' \
               + 'in_features=' + str(self.in_features) \
               + ', out_features=' + str(self.out_features) \
               + ', s=' + str(self.s) \
               + ', m=' + str(self.m) + ')'

# Define the input images
image1 = torch.randn(1, 3, 112, 112)  # Example size, adjust as needed
image2 = torch.randn(1, 3, 112, 112)
image3 = torch.randn(1, 3, 112, 112)
image4 = torch.randn(1, 3, 112, 112)

# Initialize your MarginCosineProduct model
model = MarginCosineProduct(in_features=112, out_features=4)  # Adjust parameters as needed
 # Adjust parameters as needed

# Assuming you have labels for each image, pass them through the model
label1 = torch.tensor([0])  # Example label, adjust as needed
label2 = torch.tensor([1])
label3 = torch.tensor([2])
label4 = torch.tensor([3])

output1 = model(image1, label1)
output2 = model(image2, label2)
output3 = model(image3, label3)
output4 = model(image4, label4)

# Now you have the outputs for all four images
print("Output 1:", output1)
print("Output 2:", output2)
print("Output 3:", output3)
print("Output 4:", output4)


Output 1: tensor([[[[ -49.9987,  -60.5758,   -5.6720,   38.7655],
          [ -19.8336,   34.4047,  -40.8766,   -9.6081],
          [   9.5158,   26.4086,   49.7799,   21.0002],
          ...,
          [ -18.0481,   25.6533,   46.4495,  -29.3534],
          [ -48.2325,   56.3370,   43.1090,  -20.3454],
          [ -59.8997,    0.5422,    1.0257,  -10.5587]],

         [[  -7.0083,   19.7323,    7.0951,  -40.5144],
          [-101.9637,   16.8854,  -23.4937,   -4.6519],
          [  -2.3915,   50.7866,   -7.5875,   -8.0136],
          ...,
          [ -62.0186,   42.8068,   13.1071,   -1.9307],
          [ -39.0050,    7.0462,  -10.1031,  -32.4727],
          [ -10.3053,    5.0290,   38.0453,  -50.8202]],

         [[-129.8317,  -39.7528,  -50.3780,   20.4699],
          [  16.2314,   -6.0088,   11.6122,  -94.9623],
          [ -42.0577,   63.0859,   19.8126,   -5.8765],
          ...,
          [ -36.8624,    2.1301,    2.8408,   -5.6224],
          [ -71.2068, -122.4586,   -6.2787,  